# Module 4: AprilTags

Our camera is now able to detect images and perform basic modifications to the frame captured. With this, we will now use a library to detect specific markers known as "AprilTags". They are similar to QR codes, in that they contain information using black and white squares on a 2D image (such as a paper), but are much smaller, to improve detectability and robustness.

This module should follow Module 3: Camera and Computer Vision Basics

Here are the normal packages we need to import in our code

In [ ]:
# pip3 install --upgrade pip
# pip3 install dt-apriltags

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output, Markdown
import ipywidgets as widgets
import time

robot = Robot()
camera = Camera.instance()

Now we need a new one, known as dt_apriltags! Additionally, we must create a "detector" object that will be used to analyze the image taken from the camera to find any april tags.

In [ ]:
import dt_apriltags

detector = dt_apriltags.Detector()

This following loop will detect AprilTags located in the frame of the camera, draw a box around the tag detected, and display the AprilTag's ID. The individual lines will be explored in the block after the code.

In [ ]:
# Detection Loop
while True:
    clear_output(wait = True)
    image = np.array(camera.value)
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    detections = detector.detect(gray_frame)

    for detection in detections:
        corners = detection.corners
        corners = [(int(x), int(y)) for (x, y) in corners]

        cv2.polylines(image, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)

        tag_id = detection.tag_id
        cv2.putText(image, f"ID: {tag_id}", (int(corners[0][0]), int(corners[0][1])-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    display(Image(data=bgr8_to_jpeg(image)))
    time.sleep(0.05)


To give you a sense of the various code in this block, here is an explanation walking through each line.

<i>
<code>clear_output(wait = True)</code>

<code>image = np.array(camera.value)</code>

<code>gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)</code>
</i>

The first few lines are similar to what we accomplished in Module 3. We take the image captured by the camera and convert it to a grayscale image.

<i><code>detections = detector.detect(gray_frame)</i></code>

This line is the main purpose of the AprilTags library we imported. The "detector", which we created above when we first imported the package, is an object/class. The "detect" is a method within this class that takes an image and returns a list of april tags detected in the image. Each AprilTag is stored in its own custom format, with features that define both what it is and where it is (this will be shown later).

<i><code>for detection in detections:</i></code>

Since the return is a list of AprilTags detected, we can go through each detected AprilTag in this for loop.

<i><code>corners = detection.corners</i></code>

<i><code>corners = [(int(x), int(y)) for (x, y) in corners]</i></code>

One of the specific ways an AprilTag is stored is with locating where the four corners are within the image frame. These two lines save those corner values, and then convert them into integers (just for ease of imaging). It is important to note that they are stored as purely x and y coordinates within the frame of the image, and not in the actual world. Therefore, these values refer to the specific pixels of the image for their location.

<i><code>cv2.polylines(image, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)</i></code>

This line uses cv2 to create a box that can be seen visually when we look at the image within our jupyter notebook.

<i><code>tag_id = detection.tag_id</i></code>

<i><code>cv2.putText(image, f"ID: {tag_id}", (int(corners[0][0]), int(corners[0][1])-10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)</i></code>

This is one of the other pieces of data stored within the AprilTag. Because there are different patterns that an AprilTag can encompass, they each have their own unique ID to denote which pattern is which. If you try different AprilTags, you will notice they all have their own unique IDs that show on the image. The function cv2.putText is used to display these values on the image so we can see visually what tag it is finding.

<i><code>display(Image(data=bgr8_to_jpeg(image)))</i></code>

<i><code>time.sleep(0.05)</i></code>

Same as in Module 3, this converts the image to jpeg and displays, while the time.sleep sets the framerate.

### Estimating Tag Pose

Now we want to determine where that AprilTag is located with respect to the camera. To accomplish this, we need two sets of data: our camera's intrinsic parameters and the size of the AprilTag (in meters). Luckily we already have the intrinsic parameters from Module 3, we just need to measure the size of the AprilTag.

Once you get your data, enter it into the code below.

In [ ]:
# TODO: Update these values below
cam_mtx = [94.861, 119.511, 119.506, 114.969]
AT_size = .065

Now we will use the same code as above, but with a slight tweak. When using the detect method, as in:

> detections = detector.detect(gray_frame)

there are other parameters that can be inputted that are by default set to Nones. But these will allow us to actually detect the position or pose of the AprilTag. From the dt_apriltags library documentation, the function header for the detect method appears as so:

> def detect(self, img, estimate_tag_pose=False, camera_params=None, tag_size=None):

To estimate the tag_pose, we must set the parameter to "True", set the camera_params to the intrinsic parameters we entered above, and the tag_size to the measurements we made. Nowwe can see the translation and rotation matrices which tell us the position and orientation of the AprilTag in relation to the camera (and if we know the world position and orientation of the AprilTag, we can find the position and orientation of the Jetbot itself).

In [ ]:
# Creating Widgets for displaying output
image_widget = widgets.Image(format='jpeg')
tag_label = widgets.Label()
pose_t_output = widgets.Output()
pose_R_output = widgets.Output()

display(widgets.HBox([image_widget, tag_label, pose_t_output, pose_R_output]))

while True:
    clear_output(wait = True)
    image = np.array(camera.value)
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # TODO: Write this line with the necessary parameters from the function header shown above
    detections = detector.detect(gray_frame, True, cam_mtx, AT_size)

    for detection in detections:
        corners = detection.corners
        corners = [(int(x), int(y)) for (x, y) in corners]

        cv2.polylines(image, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)

        tag_id = detection.tag_id
        cv2.putText(image, f"ID: {tag_id}", (int(corners[0][0]), int(corners[0][1])-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    image_widget.value = bgr8_to_jpeg(image)

    # Getting Pose Values from AprilTag
    if detections:
        tag_label.value = f"**Last Detected ID:** {tag_id}"
        pose_t_str = np.array2string(detections[0].pose_t, precision=3, suppress_small=True)
        pose_R_str = np.array2string(detections[0].pose_R, precision=3, suppress_small=True)
        with pose_t_output:
            pose_t_output.clear_output(wait=True)
            print("Translation Pose (t):")
            print(pose_t_str)
        with pose_R_output:
            pose_R_output.clear_output(wait=True)
            print("Rotation Pose (R):")
            print(pose_R_str)
        
    time.sleep(0.05)

Above this, you should see the camera feed from the <b><span style="color:#154734">JetBot</span></b>, as well as the Tag ID and the matrices for both translation and rotation. The translation matrix should be in the form of [x, y, z].

Now using a ruler, place the AprilTag 10cm away from the front of the camera, and record the z value as determined by the detector. Repeat these steps again for 20cm and 30cm as well.